# Calculating trigger times for JHS3

Import modules

In [154]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import scipy.stats as stats
from scipy.stats import zscore
from statistics import mean

Set up data

In [306]:
participantNumber = [10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                     21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                     31, 32, 33, 34, 35]


blockNumber = [1, 2, 3, 4]
sample = ["Silk", "Hessian"]
    
freq = 20 # in Hz
timeSteps = 1/freq

baselineLen, primeLen, trialLen = 4, 1, 4

baselineInterval, primeInterval, trialInterval = baselineLen*freq, primeLen*freq, trialLen*freq
baselinePrimeInterval, totalTrialInterval = baselineInterval+primeInterval, trialInterval+baselinePrimeInterval

maxSampleRejection = 10 #value is percent
minimumSampleWholeTrial, minimumSampleTrial, minimumSampleBaseline = totalTrialInterval-((totalTrialInterval/100)*maxSampleRejection), (trialInterval/100)*maxSampleRejection, (baselineInterval/100)*maxSampleRejection

rejectTrigger, acceptTrigger = 903, 72

SD = 2

Read compilation table

In [233]:
path = r"C:\Users\jlh94\FORCE PLATE SOFTWARE (21Nov2020)\JHS3_Processed_100Hz\\"
fileName  = "BlockAveragesCompilation(" 
fileType = ".csv"

df = pd.read_csv(path + fileName + str(freq) + '.0Hz)' + fileType)

df.rename(columns={ df.columns[6]: "Study_Block"}, inplace = True)

Keep only relevent columns

In [293]:
df = df.rename(columns={ 'X_Corr (mm)': "X_Corr", 'Y_Corr (mm)': 'Y_Corr', 
                        'Velocity_X (mm/s)': 'Velocity_X', 'Velocity_Y (mm/s)': 'Velocity_Y',
                       'Speed (mm/s)': 'Speed', 'Friction (N)': 'Friction'})
df = df[['Participant', 'Study_Block','Sample', 'Time (s)','Trigger', 
         'FzLocallyFinite', 'Speed', 'Friction']]
# 'X_Corr', 'Y_Corr','Velocity_X',Velocity_Y', 

Loop through data to identify x co-ordinate where new trigger will be

In [329]:
dfTriggerRelTime = pd.DataFrame() #create an empty dataframe where we will append our data
dfERDTriggers = pd.DataFrame()

for p in participantNumber:
    dfP = df[df['Participant'] == p]  # when value in 'Participant' column is equal to the participant value in loop
    for b in blockNumber:
        dfB = dfP[dfP['Study_Block'] == b] # when value in 'Study_block' column is equal to the block value in loop
        for s in sample:
            dfS = dfB[dfB['Sample'] == s].reset_index(drop=True)# when value in 'Sample' column is equal to the smaple value in loop
            
            num =[] # create empty array
            for ind in dfS.index: #loop through the data frame
                if dfS.Trigger[ind] > 1.3: # identify instances when the trigger value is greater than 1.3
                    if  len(num) == 0: # if the num list is empty
                        num.append(ind) # appened the index value to the empty array that was previously created 
                    else: # if the num list is not empty
                        if ind>(num[-1]+trialInterval): # if the ind value is greater than the last num value plus the time interval of the trial in samples
                            num.append(ind) # appened the index value to the empty array that was previously created 
                            
            x = pd.DataFrame()
            for n in num: 
                dfEpoch= dfS[n-baselinePrimeInterval:n+trialInterval].reset_index(drop=True)
                dfFiltered = dfEpoch[dfEpoch['FzLocallyFinite'] == 1] # only look at the rows where 'FzLocallyFinite' is equal to 1, not 0 
                meanBaseline, meanTrial = dfFiltered[0:baselineInterval].Speed.mean(), dfFiltered[baselinePrimeInterval:totalTrialInterval].Speed.mean()
                x = x.append({'meanBaseline': meanBaseline, 'meanTrial': meanTrial},ignore_index=True)
            
            x['baselineZscore'], x['trialZscore'] = abs(stats.zscore(x.meanBaseline)), abs(stats.zscore(x.meanTrial))
            x.loc[x.baselineZscore >= SD, 'meanBaseline'] = np.nan
            x.loc[x.trialZscore >= SD, 'meanTrial'] = np.nan
            baselineMax, trialMin = x.meanBaseline.max(),x.meanTrial.min()
            
            for e in range(len(num)): 
                dfEpoch= dfS[num[e]-baselinePrimeInterval:num[e]+trialInterval].reset_index(drop=True)
                dfEpoch.insert(3, "RelativeTime", np.arange(start=(minusTriggerTime*-1), stop=trialLen, step=timeSteps), True) # insert a column with the relative time from the trigger
                dfEpoch.insert(3, "Trial", e+1) # insert a column with the relative time from the trigger
                dfFiltered = dfEpoch[dfEpoch['FzLocallyFinite'] == 1] # only look at the rows where 'FzLocallyFinite' is equal to 1, not 0 
                meanBaseline, meanTrial = dfFiltered[0:baselineInterval].Speed.mean(), dfFiltered[baselinePrimeInterval:totalTrialInterval].Speed.mean()
                dfFiltered = dfFiltered.drop(columns=["FzLocallyFinite", "Time (s)"])
                
                #The loop below rejects trials where there are fewer than 75% of samples from baseline start to the end of the trial were there was sufficient force to perform calculations
                if ((len(dfFiltered)) < minimumSampleWholeTrial): #skip over dataframes that have less than 75% of samples
                    dfTriggerRelTime = dfTriggerRelTime.append({'Participant': p, 'Study_Block': b, 'Sample': s, 'Trial': (e+1),'Relative Time': np.nan,
                                                'Trigger': rejectTrigger, 'Speed': np.nan, 'Friction': np.nan}
                                               ,ignore_index=True)
                    dfERDTriggers = dfERDTriggers.append({'Participant': p, 'Study_Block': b, 'Sample': s, 'Trial': (e+1),
                                                          'Trigger': rejectTrigger},ignore_index=True)
                    
                elif meanBaseline > baselineMax or meanTrial<trialMin:
                    dfTriggerRelTime = dfTriggerRelTime.append({'Participant': p, 'Study_Block': b, 'Sample': s, 'Trial': (e+1),'Relative Time': np.nan,
                                                'Trigger': rejectTrigger, 'Speed': np.nan, 'Friction': np.nan}
                                               ,ignore_index=True)
                                                                
                    dfERDTriggers = dfERDTriggers.append({'Participant': p, 'Study_Block': b, 'Sample': s, 'Trial': (e+1),
                                                'Trigger': rejectTrigger},ignore_index=True)
                    
                else: 
                    dfTriggerRelTime = dfTriggerRelTime.append(dfFiltered)
                    dfERDTriggers = dfERDTriggers.append({'Participant': p, 'Study_Block': b, 'Sample': s, 'Trial': (e+1),
                                                'Trigger': acceptTrigger},ignore_index=True)


In [330]:


#print(dfTriggerRelTime)
print(dfERDTriggers)

     Participant   Sample  Study_Block  Trial  Trigger
0           10.0     Silk          1.0    1.0     72.0
1           10.0     Silk          1.0    2.0     72.0
2           10.0     Silk          1.0    3.0     72.0
3           10.0     Silk          1.0    4.0     72.0
4           10.0     Silk          1.0    5.0     72.0
5           10.0     Silk          1.0    6.0    903.0
6           10.0     Silk          1.0    7.0     72.0
7           10.0     Silk          1.0    8.0     72.0
8           10.0     Silk          1.0    9.0     72.0
9           10.0     Silk          1.0   10.0     72.0
10          10.0     Silk          1.0   11.0     72.0
11          10.0     Silk          1.0   12.0     72.0
12          10.0     Silk          1.0   13.0     72.0
13          10.0     Silk          1.0   14.0     72.0
14          10.0     Silk          1.0   15.0     72.0
15          10.0     Silk          1.0   16.0     72.0
16          10.0     Silk          1.0   17.0     72.0
17        